In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import requests as rq
import bs4
from bs4 import BeautifulSoup

## Part. 1
**Create a python file that webscrapes GDP by country and plots a stacked interactive bar plot using plotly. Stack countries within regions using the IMF numbers. Please include this in your ipython notebook and output your plot to an html file containing the plot.**

First, I grab table for GDP by country from the given website

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
page = rq.get(url)

bs4page = bs4.BeautifulSoup(page.text, 'html.parser')
tables = bs4page.find('table',{'class':"wikitable"})

gdp = pd.read_html(str(tables))
gdp = pd.read_html(str(tables))[0]

gdp = gdp.dropna()
gdp.head()

<ipython-input-3-39c2b5d6e3d2>:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  gdp = pd.read_html(str(tables))
<ipython-input-3-39c2b5d6e3d2>:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  gdp = pd.read_html(str(tables))[0]


Country/Territory IMF[1][12]            World Bank[13]             \
  Country/Territory   Forecast       Year       Estimate       Year   
0             World  115494312       2025      105435540       2023   
1     United States   30337162       2025       27360935       2023   
2             China   19534894  [n 1]2025       17794782  [n 3]2023   
3           Germany    4921563       2025        4456081       2023   
4             Japan    4389326       2025        4212945       2023   

  United Nations[14]             
            Estimate       Year  
0          100834796       2022  
1           25744100       2022  
2           17963170  [n 1]2022  
3            4076923       2022  
4            4232173       2022

I dropped the first row to convert from Multiindex title to single index. Then, merge with publically available data to fine the region for each country.

In [4]:
# drop the first row to remove multiindex title
gdp.columns =gdp.columns.droplevel(0)
print(gdp.columns)

# get list of countires by regions
region = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv")
region.head()

# merge with region dataset to find the region for each country
#gdp_region = pd.merge(gdp, region, left_on = gdp['Country/Territory', 'Country/Territory'], right_on = region['name'])
gdp_region = pd.merge(gdp, region[['name', 'region']], left_on='Country/Territory', right_on='name')
gdp_region.head()

Index(['Country/Territory', 'Forecast', 'Year', 'Estimate', 'Year', 'Estimate',
       'Year'],
      dtype='object')


,Country/Territory,Forecast,Year,Estimate,Year,Estimate,Year,name,region
0,China,19534894,[n 1]2025,17794782,[n 3]2023,17963170,[n 1]2022,China,Asia
1,Germany,4921563,2025,4456081,2023,4076923,2022,Germany,Europe
2,Japan,4389326,2025,4212945,2023,4232173,2022,Japan,Asia
3,India,4271922,2025,3549919,2023,3465541,2022,India,Asia
4,France,3283429,2025,3030904,2023,2775316,2022,France,Europe


In [8]:
gdp_region = gdp_region.assign(gdp_number = gdp['Forecast'], country= gdp['Country/Territory'])
print(gdp_region.columns)

# drop the 'World' row
print(gdp_region[gdp_region['country'] == 'World'])
gdp_region = gdp_region[gdp_region['country'] != 'World']

Index(['Country/Territory', 'Forecast', 'Year', 'Estimate', 'Year', 'Estimate',
       'Year', 'name', 'region', 'gdp_number', 'country'],
      dtype='object')
  Country/Territory  Forecast       Year  Estimate       Year  Estimate  \
0             China  19534894  [n 1]2025  17794782  [n 3]2023  17963170   

        Year   name region gdp_number country  
0  [n 1]2022  China   Asia  115494312   World  


I plotted stack bar plot to present the total GDP sum across regions by using plotly.

In [10]:
fig = px.bar(gdp_region, x = "region", y = "gdp_number", color = "country", title='Global GDP by Country and Region',
                 labels={'GDP': 'GDP in Millions (USD)'}, barmode='stack')
fig.write_html("stacked_bar.html")
fig.show()

## Part. 2
**Look at the chapter on interactive graphics and, specifically, the code to display a subject's MRICloud data as a sunburst plot. Do the following. Display this subject's data as a Sankey diagram. Display as many levels as you can (at least 3) for Type = 1, starting from the intracranial volume.**

In [11]:
dat = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21.csv").drop(['Unnamed: 0'], axis = 1)
dat.head()

dat = dat.assign(id_char = dat.id.astype(str))
fig = px.bar(dat, x = "id_char", y = "volume", color = "roi")
fig.show()

icv = dat.groupby(['id']).volume.sum().reset_index().rename(columns = {'volume' : 'icv'})
dat = pd.merge(dat, icv, on = 'id')
dat = dat.assign(comp = dat.volume / dat.icv)
dat.head()

## load in the hierarchy information
url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi",
    "modify.1" : "level4",
    "modify.2" : "level3",
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
multilevel_lookup.head()

,roi,level4,level3,level2,level1
0,SFG_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L
1,SFG_R,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R
2,SFG_PFC_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L
3,SFG_PFC_R,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R
4,SFG_pole_L,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L


In [12]:
## Now load in the subject data
id = 127
subjectData = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv")
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]
## Merge the subject data with the multilevel data
subjectData = pd.merge(subjectData, multilevel_lookup, on = "roi")
subjectData = subjectData.assign(icv = "ICV")
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))
subjectData.head()

,roi,volume,level4,level3,level2,level1,icv,comp
0,SFG_L,12926,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009350
1,SFG_R,10050,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.007270
2,SFG_PFC_L,12783,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009247
3,SFG_PFC_R,11507,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.008324
4,SFG_pole_L,3078,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.002227


I plotted sunburst plot to display a subject's MRICloud data by using plotly.

In [14]:
fig = px.sunburst(subjectData, path=['icv', 'level1', 'level2', 'level3', 'level4', 'roi'],
                  values='comp', width=800, height=800)
fig.write_html("sankey.html")
fig.show()